In [1]:
# Multiple horizons MTM analysis
include("C:\\Users\\Federico\\Documents\\GitHub\\pysical_locomotion\\analysis\\behavior\\analysis_fixtures.jl")
include("C:\\Users\\Federico\\Documents\\GitHub\\pysical_locomotion\\analysis\\behavior\\analysis_visuals.jl")
Pkg.add("IterTools")
import jcontrol as jc
using Printf
import jcontrol.control: default_control_options
import jcontrol.bicycle: Bicycle
import IterTools: product as ×
import jcontrol: Track, toDict
import InfiniteOpt: termination_status

import jcontrol.Run: run_mtm

@Info (Main):  Loaded 995 trials 
  ╰──────────────────────────────────────────────── 
                      Mon, 30 May 2022 13:52:11 
@Info (Main):  After tortuosity analysis, discarded 8.643% of trials | 909 
  │                trials left 
  ╰──────────────────────────────────────────────── 
                      Mon, 30 May 2022 13:52:14 


  Activating project at `C:\Users\Federico\Documents\GitHub\pysical_locomotion\analysis\behavior`
    Updating registry at `C:\Users\Federico\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\Federico\Documents\GitHub\pysical_locomotion\analysis\behavior\Project.toml`
  No Changes to `C:\Users\Federico\Documents\GitHub\pysical_locomotion\analysis\behavior\Manifest.toml`


# MTM on track
Solve the MTM prolblem on a bunch of tracks and with slightly different models paramters to created data to use for training RNNs.

## Create bikes with different parameters

In [2]:
default_bike = Bicycle()
variables = (:l_f, :l_r, :width, :c)

σ = .05
n = 3

defaults = Dict(v=>getfield(default_bike, v) for v in variables)
variables_values = Dict(
    v => collect(range(defaults[v]-σ*defaults[v], stop=defaults[v]+σ*defaults[v], length=n)) for v in variables
)

n_bikes = length(collect(×(values(variables_values)...)))

81

## Load tracks

In [3]:
# fld = "/Users/federicoclaudi/Documents/Github/LocomotionControl/analysis/RNN/tracks"
fld = "C:\\Users\\Federico\\Documents\\GitHub\\pysical_locomotion\\analysis\\RNN\\tracks"
tracks_files = glob("F1_*.npy", fld)
n_tracks = length(tracks_files)

25

In [4]:
N = length(
    ×(
        values(variables_values)...,
        tracks_files
    )
)

dur_h = round(N / 60; digits=2)

print(
    "Tot number of simulations = $N || Duration $(dur_h) hours"
)

Tot number of simulations = 2025 || Duration 33.75 hours

## Simulate all

In [5]:
tname(trackfile) = splitext(splitpath(trackfile)[end])[1]
_tracks = Dict(tname(f) => Track(; start_waypoint=4, npyfile=f, const_width=true, track_length=nothing) for f in tracks_files)


Dict{String, Track} with 25 entries:
  "F1_IMS"           => Track. 1996.0 cm long, 100001 waypoints.
  "F1_Sakhir"        => Track. 2685.0 cm long, 100001 waypoints.
  "F1_Silverstone"   => Track. 2926.0 cm long, 100001 waypoints.
  "F1_YasMarina"     => Track. 2753.0 cm long, 100001 waypoints.
  "F1_Spa"           => Track. 3483.0 cm long, 100001 waypoints.
  "F1_Spielberg"     => Track. 2141.0 cm long, 100001 waypoints.
  "F1_MoscowRaceway" => Track. 2013.0 cm long, 100001 waypoints.
  "F1_Shanghai"      => Track. 2704.0 cm long, 100001 waypoints.
  "F1_Sochi"         => Track. 2903.0 cm long, 100001 waypoints.
  "F1_Norisring"     => Track. 1131.0 cm long, 100001 waypoints.
  "F1_Suzuka"        => Track. 2885.0 cm long, 100001 waypoints.
  "F1_Sepang"        => Track. 2751.0 cm long, 100001 waypoints.
  "F1_MexicoCity"    => Track. 2130.0 cm long, 100001 waypoints.
  "F1_Monza"         => Track. 2878.0 cm long, 100001 waypoints.
  "F1_SaoPaulo"      => Track. 2135.0 cm long, 100001

In [6]:
# save_fld = "/Users/federicoclaudi/Dropbox (UCL)/Rotation_vte/Locomotion/analysis/RNN/tracks_mtm_sims"
save_fld = "D:\\Dropbox (UCL)\\Rotation_vte\\Locomotion\\analysis\\RNN\\tracks_mtm_sims"


i = 0
for lf in variables_values[:l_f], lr in variables_values[:l_r], w in variables_values[:width], c in variables_values[:c]
    bike = Bicycle(
        l_f=lf,
        l_r=lr,
        width=w,
        c=c
    )
    i += 1  


    for trackfile in tracks_files
        track_name = tname(trackfile)
        savepath = joinpath(save_fld, "$(track_name)_bike_$i.csv")
        isfile(savepath) && begin
            # @info "skipping $track_name with bike i"
            continue
        end
        println("Doing track $(track_name)")
        track = _tracks[track_name]

        _, _, control_model, solution = run_mtm(
            :dynamics,  # model type
            2.0;  # supports density
            showtrials=nothing,
            showplots=false,
            quiet=false,
            bike=bike,
            track=track,
            n_iter=100000,
        )

        if "LOCALLY_SOLVED" == string(termination_status(control_model))
            data = DataFrame(toDict(solution))
            CSV.write(savepath, data)
        else
            @warn "FAILED" termination_status(control_model) track_name bike
        end
    end
end
